In [16]:
from sigma_inv import sigma_inv_approx
from dataset import Dataset





In [18]:
import numpy as np

μ_0 = np.array([0.75,0.25])
cov_0 = np.array([[0.1, 0.05], [0.05, 0.1]])

μ_1 = np.array([0.25, 0.75])
cov_1 = np.array([[0.01, 0.005], [0.005, 0.01]])

gamma_prior_cov = np.array([0.1])

ν = cov_0[1,1]


# Number of samples to generate
n_samples = 1000

# Generate samples alternately
samples = np.zeros((n_samples, 2))
for i in range(n_samples):
    if i % 2 == 0:
        samples[i] = np.random.multivariate_normal(μ_0, cov_0)
    else:
        samples[i] = np.random.multivariate_normal(μ_1, cov_1)


ds = Dataset(samples, emb_dim=2, N=1000, K=2)

for i in range(0,len(ds.z_vars)):
    ds.z_vars[i].probs = [1.0, 0.0] if i % 2 == 0 else [0.0, 1.0]

assumed_dof = 5 #= d+3

ds.means_vars[0].prior_cov = cov_0
ds.means_vars[0].mean = μ_0
ds.means_vars[0].cov = cov_0

ds.means_vars[1].prior_cov = cov_1
ds.means_vars[1].mean = μ_1
ds.means_vars[1].cov = cov_1


ds.gamma_vars[0].prior_cov = np.array([gamma_prior_cov / np.sqrt(ν)])
ds.gamma_vars[0].mean = np.array([cov_0[0, 1]]) / np.sqrt(ν)
ds.gamma_vars[0].cov = np.array([gamma_prior_cov / ν])
ds.gamma_vars[0].nu = cov_0[-1,-1]

ds.gamma_vars[1].prior_cov = np.array([gamma_prior_cov / np.sqrt(ν)])
ds.gamma_vars[1].mean = np.array([cov_1[0, 1]]) / np.sqrt(ν)
ds.gamma_vars[1].cov = np.array([gamma_prior_cov / ν])
ds.gamma_vars[1].nu = cov_1[-1,-1]

ds.sigma_star_vars[0].prior_scale = np.array([cov_0[0,0] * (assumed_dof - ds.d)])
ds.sigma_star_vars[0].dof = 5
ds.sigma_star_vars[0].prior_dof = 5
ds.sigma_star_vars[0].scale = np.array([cov_0[0,0] - ds.gamma_vars[0].mean ** 2])
ds.sigma_star_vars[0].nu = cov_1[-1,-1]                


ds.sigma_star_vars[1].prior_scale = np.array([cov_1[0,0] - ds.gamma_vars[0].mean ** 2])
ds.sigma_star_vars[1].dof = 5
ds.sigma_star_vars[1].prior_dof = 5
ds.sigma_star_vars[1].scale = np.array([[cov_1[0,0] * (assumed_dof - ds.d)]])
ds.sigma_star_vars[1].nu = cov_1[-1,-1]

In [36]:
sig_star_0 = ds.sigma_star_vars[0]
gamma_0 = ds.gamma_vars[0]

sigma_inv_approx(sig_star_0, gamma_0)


array([[-6.53687500e-02,  9.48683298e-04],
       [ 9.48683298e-04,  2.98035000e+00]])

In [34]:
print(f"==>> cov_0: {cov_0}")

cov_0_inv = np.linalg.inv(cov_0)

print(f"==>> cov_0_inv: {cov_0_inv}")

==>> cov_0: [[0.1  0.05]
 [0.05 0.1 ]]
==>> cov_0_inv: [[13.33333333 -6.66666667]
 [-6.66666667 13.33333333]]


In [13]:
import numpy as np
from scipy.stats import invwishart

def generate_invwishart_samples(df, scale, size):
    """
    Generate random samples from an inverse Wishart distribution.
    
    Parameters:
    df (int): Degrees of freedom.
    scale (ndarray): Scale matrix (positive definite).
    size (int): Number of samples to generate.
    
    Returns:
    ndarray: Generated samples.
    """
    samples = invwishart.rvs(df, scale, size=size)
    return samples

def compute_second_moment(samples):
    """
    Compute the second moment (mean of the squares of the samples).
    
    Parameters:
    samples (ndarray): Samples from the inverse Wishart distribution.
    
    Returns:
    ndarray: Second moment matrix.
    """
    # We need to calculate the second moment, which is the expectation of the squares of the samples
    print(f"samples[0]: {samples[0]}")
    second_moment = np.mean([np.matmul(sample, sample) for sample in samples], axis=0)
    return second_moment

# Parameters for the inverse Wishart distribution
df = 5  # degrees of freedom
scale = np.array([[1, 0.1], [0.1, 1]])  # scale matrix
num_samples = 100_000  # number of samples to generate

# Generate samples
samples = generate_invwishart_samples(df, scale, num_samples)

# Compute the second moment
second_moment = compute_second_moment(samples)

print("Approximated second moment matrix:")
print(second_moment)


samples[0]: [[ 0.24957708 -0.01494744]
 [-0.01494744  0.14770534]]
Approximated second moment matrix:
[[ 2.23253595 -1.03001724]
 [-1.03001724  3.47101936]]


In [15]:
from sigma_dist import Sigma_Star

S = Sigma_Star(0,2)

S.scale = np.array([[1, 0.1], [0.1, 1]])
S.dof = 5

S.second_mom()

array([[ 0.59772812, -0.08502534],
       [-0.08502534,  0.59772812]])

In [17]:
import numpy as np

def inverse_wishart_mean(scale_matrix, degrees_of_freedom):
    """
    Calculate the mean of an Inverse Wishart distribution if it is defined.

    Parameters:
    scale_matrix (numpy.ndarray): The scale matrix (Psi).
    degrees_of_freedom (float): The degrees of freedom (nu).

    Returns:
    numpy.ndarray: The mean of the Inverse Wishart distribution.
    """
    p = scale_matrix.shape[0]
    if degrees_of_freedom <= p + 1:
        raise ValueError("Degrees of freedom must be greater than the dimension of the scale matrix plus one.")
    
    mean = scale_matrix / (degrees_of_freedom - p - 1)
    return mean

# Example usage:
scale_matrix = np.array([[1, 0.5], [0.5, 2]])
degrees_of_freedom = 5

mean = inverse_wishart_mean(scale_matrix, degrees_of_freedom)
print("Mean of the Inverse Wishart distribution:\n", mean)


Mean of the Inverse Wishart distribution:
 [[0.5  0.25]
 [0.25 1.  ]]


[[0.025]]
[0.15811388]
